In [1]:
pip install --upgrade qiskit qiskit-machine-learning qiskit-algorithms numpy matplotlib scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from qiskit.primitives import StatevectorSampler 
from qiskit_machine_learning.kernels import FidelityQuantumKernel  
from qiskit.circuit.library import ZZFeatureMap
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons

# ✅ Step 1: Custom Fidelity Function for Quantum Kernel
def custom_fidelity(sampler, circuit_1, circuit_2):
    """
    Compute the fidelity between two quantum states using the StatevectorSampler primitive.
    """
    combined_circuit = circuit_1.compose(circuit_2.inverse())  # Combine the circuits
    job = sampler.run([combined_circuit])  
    result = job.result()
    probabilities = result.quasi_dists[0]

    # Fidelity is the probability of measuring the all-zero state
    fidelity = probabilities.get(0, 0)  
    return fidelity

# ✅ Step 2: Generate Synthetic Dataset (Two-Moons Dataset)
X, y = make_moons(n_samples=100, noise=0.1, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ✅ Step 3: Define Quantum Feature Map
feature_map = ZZFeatureMap(feature_dimension=2, reps=2, entanglement='linear')

# ✅ Step 4: Initialize the Updated Sampler
sampler = StatevectorSampler() 

# ✅ Step 5: Define Fidelity Quantum Kernel with Custom Fidelity Function
class CustomFidelityQuantumKernel(FidelityQuantumKernel):
    def __init__(self, feature_map, sampler):
        super().__init__(feature_map=feature_map, fidelity=None) 
        self.sampler = sampler

    def _compute_fidelity(self, circuit_1, circuit_2):
        return custom_fidelity(self.sampler, circuit_1, circuit_2)

# ✅ Step 6: Compute the Quantum Kernel Matrix
quantum_kernel = CustomFidelityQuantumKernel(feature_map=feature_map, sampler=sampler)

X_train_kernel = quantum_kernel.evaluate(X_train)
X_test_kernel = quantum_kernel.evaluate(X_test, X_train)

# ✅ Step 7: Train a Classical Support Vector Machine (SVM) with the Quantum Kernel
svm = SVC(kernel="precomputed")  
svm.fit(X_train_kernel, y_train)

# ✅ Step 8: Evaluate Model Accuracy
y_pred = svm.predict(X_test_kernel)

accuracy = np.mean(y_pred == y_test)
print(f"Quantum Kernel SVM Accuracy: {accuracy * 100:.2f}%")



Quantum Kernel SVM Accuracy: 60.00%
